In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount("/drive")
%cd /drive/My\ Drive/Colab\ Notebooks/STAI

Mounted at /drive
/drive/My Drive/Colab Notebooks/STAI


In [8]:
data = pd.read_csv('game_log.csv')
data

,head_x,head_y,tail_x,tail_y,food_x,food_y,direction,snake_length,score,head_top_distance,...,moves_since_eating,total_moves,closest_body_distance,enclosed_spaces_count,largest_enclosed_space,tail_reachable,food_eaten,game_ended,time_elapsed,action
0,16,10,15,10,6,19,1,2,1,10,...,45,45,1.0,0,0,True,False,False,4.692,UP
1,16,7,16,8,6,19,1,2,1,7,...,48,48,1.0,0,0,True,False,False,4.970,UP
2,16,3,16,4,6,19,1,2,1,3,...,52,52,1.0,0,0,True,False,False,5.363,UP
3,16,0,16,1,6,19,1,2,1,0,...,55,55,1.0,0,0,True,False,False,5.667,UP
4,16,15,16,16,6,19,4,2,1,15,...,80,80,1.0,0,0,True,False,False,8.108,RIGHT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7309,17,13,5,19,17,4,1,19,18,13,...,354,354,1.0,0,0,True,False,False,37.265,'UP'
7310,17,3,8,13,0,16,3,20,19,3,...,363,363,1.0,0,0,True,False,False,38.273,'LEFT'
7311,19,3,17,4,0,16,1,20,19,3,...,381,381,2.0,0,0,True,False,False,40.033,'UP'
7312,19,6,1,3,0,16,3,20,19,6,...,398,398,1.0,0,0,True,False,False,41.713,'LEFT'


In [9]:
data = data.drop(columns=['enclosed_spaces_count', 'largest_enclosed_space', 'food_eaten', 'game_ended', 'time_elapsed'])
data['closest_body_distance'] = data['closest_body_distance'].fillna(0)

data = pd.get_dummies(data, columns=['action'], prefix='', prefix_sep='')

X = data.drop(columns=['UP', 'DOWN', 'LEFT', 'RIGHT'])
y = data[['UP', 'DOWN', 'LEFT', 'RIGHT']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


147/147 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.5355 - loss: 0.7712 - val_accuracy: 0.7395 - val_loss: 0.3323
Epoch 2/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7378 - loss: 0.3056 - val_accuracy: 0.7515 - val_loss: 0.3309
Epoch 3/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7283 - loss: 0.3272 - val_accuracy: 0.7515 - val_loss: 0.4972
Epoch 4/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7084 - loss: 0.4012 - val_accuracy: 0.6746 - val_loss: 0.4391
Epoch 5/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6967 - loss: 0.4665 - val_accuracy: 0.6687 - val_loss: 0.4689


In [12]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test.values, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_test_classes, y_pred_classes, average='weighted', zero_division=1)
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted', zero_division=1)

if np.isnan(y_test.values).any() or np.isnan(y_pred).any():
    raise ValueError("Input contains NaN.")

r2 = r2_score(y_test.values, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print(f'R² Score: {r2}')

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.6691729323308271
Precision: 0.7436796397499181
Recall: 0.6691729323308271
F1-Score: 0.644440622925262
R² Score: 0.3895225628324706
